In [1]:
%display latex

In [2]:
import numpy as np
s1 = matrix(
    [
        [0, 1],
        [1, 0]
    ]
)
s2 = matrix(
    [
        [0, i],
        [-i, 0]
    ]
)
s3 = matrix(
    [
        [1, 0],
        [0, -1]
    ]
)

one = matrix.identity(s1.dimensions()[0])

s = [s1, s2, s3]
s

[
[0 1]  [ 0  I]  [ 1  0]
[1 0], [-I  0], [ 0 -1]
]

In [3]:
# Hack
def Id(self, x): return self(x)

p = vector([
    function("pi"+str(i), latex_name="\\pi_"+str(i), conjugate_func=Id)(x) for i  in range(1, 4)
])
p

(pi1(x), pi2(x), pi3(x))

In [4]:
var("e", latex_name="\\varepsilon", domain="real")
var("d", latex_name="\\delta", domain="real")
var("mu", latex_name="\\mu_I", domain="real")
var("a", latex_name="\\alpha", domain="real")

pi_s = e * sum([s[i]*p[i] for i in range(len(s))])
v = 1/2*d*mu*s3

n = 5 # expand up to, not incluing, n

In [5]:
def mat_series(mat, x, n):
    d = mat.dimensions()
    for i in range(0, d[0]):
        for j in range(0, d[1]):
            mat[i, j] = mat[i, j].series(x, n).truncate()

def mat_simp(mat):
    d = mat.dimensions()
    for i in range(0, d[0]):
        for j in range(0, d[1]):
            mat[i, j] = mat[i, j].full_simplify()
            mat[i, j] = mat[i, j].trig_reduce()

def mat_prep(mat, n=n):
    mat_series(mat, e, n)
    mat_simp(mat)
    return mat

def print_e(elem):
    coeff = elem.coefficients(e)
    for i in range(len(coeff)):
        print(e^coeff[i][1], ":")
        c = coeff[i][0].coefficients(mu)
        for k in c:
            s = k[0].full_simplify()*mu**k[1]
            s = s.trig_reduce()
            pretty_print(s.factor().factor())

In [6]:
acom = lambda A1, A2 : A1 * A2 + A2 * A1
com = lambda A1, A2 : A1 * A2 - A2 * A1

def proj(A):
    A1 = A.trace()/2
    A = A - one * A1
    v = [A1, acom(A, s1)[0, 0]/2, acom(A, s2)[0, 0]/2, acom(A, s3)[0, 0]/2]
    v = [a.full_simplify().trig_reduce() for a in v]
    return v 

In [7]:
POW = lambda A, n : matrix.identity(A.dimensions()[0]) if (n == 0) else A * POW(A, n-1)
EXP = lambda A, n : sum([POW(A, i)/factorial(i) for i in range(n+1)])

In [8]:
m = Matrix([[0, 1], [-1, 0]])
POW(m, 3)

N(EXP(-pi*m, 100)) # Matrix rep. of i. Should be -one

[    -1.00000000000000 -3.46629141516451e-16]
[ 3.46629141516451e-16     -1.00000000000000]

In [9]:
A_a = one*cos(a/2) + I * s1 * sin(a/2)
U = lambda n: EXP(I * pi_s/2, n)
SIGMA = lambda n: mat_prep(A_a * U(n) * U(n) * A_a)
S = mat_prep(SIGMA(n))

In [10]:
k = proj(S)
for i, ki in enumerate(k):
    print("tau_{}:".format(i))
    c = ki.coefficients(e)
    for ci in c:
        pretty_print(ci[0].full_simplify().factor())

tau_0:


cos(a)

-pi1(x)*sin(a)

-1/2*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*cos(a)

1/6*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*pi1(x)*sin(a)

...

I*pi2(x)

-1/6*I*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*pi2(x)

tau_3:


I*pi3(x)

-1/6*I*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*pi3(x)

In [11]:
dS = diff(S, x) # nabla_mu Sigma
dSct = diff(S.C.T, x)
COM = v*S - S*v # [v_mu, Sigma]

In [12]:
term1 = mat_prep(dS*(dSct))
term2 = I*mat_prep(COM*dSct - dS*(COM.C.T))
term3 = mat_prep(COM*COM.C.T)
term4 = mat_prep(S + S.C.T)
terms_full = [term1, term2, term3, term4]

In [13]:
terms = [t.trace().full_simplify() for t in terms_full]

for i, t in enumerate(terms):
    print("term {}:".format(i+1))
    print_e(t)

term 1:
e^2 :


2*diff(pi1(x), x)^2 + 2*diff(pi2(x), x)^2 + 2*diff(pi3(x), x)^2

e^4 :


-2/3*pi2(x)^2*diff(pi1(x), x)^2 - 2/3*pi3(x)^2*diff(pi1(x), x)^2 + 4/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 2/3*pi1(x)^2*diff(pi2(x), x)^2 - 2/3*pi3(x)^2*diff(pi2(x), x)^2 + 4/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 4/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 2/3*pi1(x)^2*diff(pi3(x), x)^2 - 2/3*pi2(x)^2*diff(pi3(x), x)^2

term 2:
e :


...

-2*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*cos(a)

e^3 :


2/3*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*pi1(x)*sin(a)

e^4 :


1/6*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)^2*cos(a)

In [14]:
var("m", domain="real")
var("f", domain="real")

L = 1/4*(term1+term2+term3+m^2*term4).trace().full_simplify().trig_reduce()
L_coeff = L.coefficients(e)
L_coeff = [c[0].full_simplify().trig_reduce() for c in L_coeff]

In [15]:
for i in range(len(L_coeff)):
    print(e^i)
    c = L_coeff[i].coefficients(mu)
    for k in c:
        s = k[0].full_simplify()*mu**k[1]
        s = s.full_simplify()
        s = s.expand()
        pretty_print(s.factor().factor())

1


m^2*cos(a)

1/2*d^2*mu^2*sin(a)^2

e


-m^2*pi1(x)*sin(a)

...

-2/3*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*d^2*mu^2*cos(a)*pi1(x)*sin(a)

e^4


1/24*m^2*cos(a)*pi1(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi2(x)^2 + 1/24*m^2*cos(a)*pi2(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi3(x)^2 + 1/12*m^2*cos(a)*pi2(x)^2*pi3(x)^2 + 1/24*m^2*cos(a)*pi3(x)^4 - 1/6*pi2(x)^2*diff(pi1(x), x)^2 - 1/6*pi3(x)^2*diff(pi1(x), x)^2 + 1/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 1/6*pi1(x)^2*diff(pi2(x), x)^2 - 1/6*pi3(x)^2*diff(pi2(x), x)^2 + 1/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 1/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 1/6*pi1(x)^2*diff(pi3(x), x)^2 - 1/6*pi2(x)^2*diff(pi3(x), x)^2

1/3*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*(pi2(x)*diff(pi1(x), x) - pi1(x)*diff(pi2(x), x))*d*mu*cos(a)

1/6*(2*pi1(x)^2*sin(a)^2 + pi2(x)^2*sin(a)^2 + pi3(x)^2*sin(a)^2 - pi1(x)^2 - pi2(x)^2)*(pi1(x)^2 + pi2(x)^2 + pi3(x)^2)*d^2*mu^2

In [16]:
L_coeff[4].coefficients(mu)[0][0].trig_reduce().factor()

1/24*m^2*cos(a)*pi1(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi2(x)^2 + 1/24*m^2*cos(a)*pi2(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi3(x)^2 + 1/12*m^2*cos(a)*pi2(x)^2*pi3(x)^2 + 1/24*m^2*cos(a)*pi3(x)^4 - 1/6*pi2(x)^2*diff(pi1(x), x)^2 - 1/6*pi3(x)^2*diff(pi1(x), x)^2 + 1/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 1/6*pi1(x)^2*diff(pi2(x), x)^2 - 1/6*pi3(x)^2*diff(pi2(x), x)^2 + 1/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 1/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 1/6*pi1(x)^2*diff(pi3(x), x)^2 - 1/6*pi2(x)^2*diff(pi3(x), x)^2

In [17]:
(((p*p)**2*cos(a)/4*m**2 - ((p*p)*(diff(p, x)*diff(p, x) ) - (diff(p, x)*p)**2))/6).expand()

1/24*m^2*cos(a)*pi1(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi2(x)^2 + 1/24*m^2*cos(a)*pi2(x)^4 + 1/12*m^2*cos(a)*pi1(x)^2*pi3(x)^2 + 1/12*m^2*cos(a)*pi2(x)^2*pi3(x)^2 + 1/24*m^2*cos(a)*pi3(x)^4 - 1/6*pi2(x)^2*diff(pi1(x), x)^2 - 1/6*pi3(x)^2*diff(pi1(x), x)^2 + 1/3*pi1(x)*pi2(x)*diff(pi1(x), x)*diff(pi2(x), x) - 1/6*pi1(x)^2*diff(pi2(x), x)^2 - 1/6*pi3(x)^2*diff(pi2(x), x)^2 + 1/3*pi1(x)*pi3(x)*diff(pi1(x), x)*diff(pi3(x), x) + 1/3*pi2(x)*pi3(x)*diff(pi2(x), x)*diff(pi3(x), x) - 1/6*pi1(x)^2*diff(pi3(x), x)^2 - 1/6*pi2(x)^2*diff(pi3(x), x)^2

In [18]:
var("dm", latex_name="\\Delta m", domain="real")
var("mm", latex_name="\\bar m", domain="real")
var("B0", latex_name="B_0", domain="real")
chi = 2*B0 * (mm * one + dm * s3)


mat_prep(chi*S.C.T - S*chi, n=3).trace()

-8*I*B0*dm*e*pi3(x)

In [19]:
(mat_prep(chi*S.C.T + S*chi, n=3).trace().factor()^2).series(e, 3).truncate().factor()

-64*(e^2*cos(a)^2*pi1(x)^2 + e^2*cos(a)^2*pi2(x)^2 + e^2*cos(a)^2*pi3(x)^2 - e^2*pi1(x)^2*sin(a)^2 + 2*e*cos(a)*pi1(x)*sin(a) - cos(a)^2)*B0^2*mm^2

In [20]:
(
    (
        mat_prep(chi*S.C.T + S*chi, n=3).trace()
    )
    *(
        term1 + term2 + term3
    ).trace()
).series(e, 3).truncate().factor()

4*(5*d^2*e^2*mu^2*cos(2*a)*cos(a)*pi1(x)^2 + 3*d^2*e^2*mu^2*cos(2*a)*cos(a)*pi2(x)^2 + 3*d^2*e^2*mu^2*cos(2*a)*cos(a)*pi3(x)^2 - 4*d^2*e^2*mu^2*pi1(x)^2*sin(2*a)*sin(a) - d^2*e^2*mu^2*cos(a)*pi1(x)^2 + d^2*e^2*mu^2*cos(a)*pi2(x)^2 - 3*d^2*e^2*mu^2*cos(a)*pi3(x)^2 + 4*d^2*e*mu^2*cos(a)*pi1(x)*sin(2*a) + 2*d^2*e*mu^2*cos(2*a)*pi1(x)*sin(a) - 8*d*e^2*mu*cos(a)^2*pi2(x)*diff(pi1(x), x) + 8*d*e^2*mu*cos(a)^2*pi1(x)*diff(pi2(x), x) - 8*d*e^2*mu*pi1(x)*sin(a)^2*diff(pi2(x), x) - 2*d^2*e*mu^2*pi1(x)*sin(a) - 2*d^2*mu^2*cos(2*a)*cos(a) + 8*d*e*mu*cos(a)*sin(a)*diff(pi2(x), x) + 2*d^2*mu^2*cos(a) + 4*e^2*cos(a)*diff(pi1(x), x)^2 + 4*e^2*cos(a)*diff(pi2(x), x)^2 + 4*e^2*cos(a)*diff(pi3(x), x)^2)*B0*mm

In [21]:
div2 = term1 + term2 + term3

In [22]:
k = ((div2).trace()^2).series(e, 3).truncate().full_simplify()
coeff = k.coefficients(mu)
coeff[0][0]

8*d^2*e^2*sin(a)^2*diff(pi1(x), x)^2 + 24*d^2*e^2*sin(a)^2*diff(pi2(x), x)^2 + 8*d^2*e^2*sin(a)^2*diff(pi3(x), x)^2

In [23]:
coeff[1][0]

-16*d^3*e^2*cos(a)*pi2(x)*sin(a)^2*diff(pi1(x), x) + 48*d^3*e^2*cos(a)*pi1(x)*sin(a)^2*diff(pi2(x), x) + 16*d^3*e*sin(a)^3*diff(pi2(x), x)

In [24]:
coeff[2][0]

-8*d^4*e^2*pi3(x)^2*sin(a)^4 + 16*d^4*e*cos(a)*pi1(x)*sin(a)^3 - 8*(4*sin(a)^4 - 3*sin(a)^2)*d^4*e^2*pi1(x)^2 - 8*(sin(a)^4 - sin(a)^2)*d^4*e^2*pi2(x)^2 + 4*d^4*sin(a)^4